In [1]:
#importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from math import sqrt

In [2]:
#Importing data
sales_data = pd.read_csv('kc_house_data.csv')

#exploring the data set 
sales_data.dtypes, sales_data.info(), sales_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

(id                 int64
 date              object
 price            float64
 bedrooms           int64
 bathrooms        float64
 sqft_living        int64
 sqft_lot           int64
 floors           float64
 waterfront         int64
 view               int64
 condition          int64
 grade              int64
 sqft_above         int64
 sqft_basement      int64
 yr_built           int64
 yr_renovated       int64
 zipcode            int64
 lat              float64
 long             float64
 sqft_living15      int64
 sqft_lot15         int64
 dtype: object,
 None,
                  id         price      bedrooms     bathrooms   sqft_living  \
 count  2.161300e+04  2.161300e+04  21613.000000  21613.000000  21613.000000   
 mean   4.580302e+09  5.400881e+05      3.370842      2.114757   2079.899736   
 std    2.876566e+09  3.671272e+05      0.930062      0.770163    918.440897   
 min    1.000102e+06  7.500000e+04      0.000000      0.000000    290.000000   
 25%    2.123049e+09  3.219500e

In [3]:
#splitting data into train and test sets
train_sales, test_sales = train_test_split(sales_data, train_size=0.80, test_size=0.20, random_state=100)
train_sales.shape, test_sales.shape

((17290, 21), (4323, 21))

In [4]:
#defining a simple linear regression function

def simple_linear_regression(input_feature, output):
  sum_x = input_feature.sum()
  sum_y = output.sum()
  sum_yx = (input_feature*output).sum()
  sum_xx = (input_feature**2).sum()
  n = float(len(output))
  slope = (sum_yx - (sum_y*sum_x)/n)/(sum_xx - (sum_x*sum_x)/n)
  intercept = (sum_y/n) - slope*sum_x/n
  return(intercept, slope)

In [5]:
sqft_intercept, sqft_slope = simple_linear_regression(train_sales['sqft_living'], train_sales['price'])
print("Sqft intercept: ", sqft_intercept)
print("Sqft slope: ", sqft_slope)

Sqft intercept:  -42628.97651509417
Sqft slope:  280.68541678774267


In [6]:
'''defining a simple function that takes parameter estimates above
and makes predictions using input from the test data set'''

def get_regression_predictions(input_feature, intercept, slope):
  predicted_output = intercept + slope*input_feature
  return(predicted_output)

In [7]:
'''defining a function to calculate the error metric RSS'''
def get_RSS(input_features, output, intercept, slope):
  y_hat = intercept + slope*input_features
  RSS = ((output-y_hat)**2).sum() 
  return(RSS)

In [8]:
Computed_RSS_sqft = get_RSS(train_sales['sqft_living'], train_sales['price'], sqft_intercept, sqft_slope)
Computed_RSS_sqft

1195070000113061.2

In [9]:
'''defining a function that takes a specific value for dependent variable
and calculates the input value that would result in the specific value'''

def inverse_regression_prediction(output, intercept, slope):
  return (output-intercept)/float(slope)

In [10]:
bedroom_intercept, bedroom_slope = simple_linear_regression(train_sales['bedrooms'], train_sales['price'])
print("Intercept for bedrooms: ", bedroom_intercept)
print("Slope for bedrooms: ", bedroom_slope)

Intercept for bedrooms:  135286.06806805916
Slope for bedrooms:  120138.3170170902


In [11]:
Computed_RSS_bedroom = get_RSS(train_sales['bedrooms'],train_sales['price'], bedroom_intercept, bedroom_slope)
Computed_RSS_bedroom

2112335899103246.5

Multiple Linear Regression 

1. Adding more than one input variable
2. Comparing models of different complexities on RSS 

Adding new features

some of the features added make no sense 
1. lat+long

while others like 

1. bedroom squared increase the difference between houses with a too few bedrooms and too many bedrooms
2. bed_bath_rooms is an interaction term 
3. log of sqft living helps de-emphasize outliers by bringing large values closer'''

In [12]:
train_sales['bedrooms_squared'],train_sales['bed_bath_rooms'],train_sales['log_sqft_living'],train_sales['lat_plus_long'] = [train_sales['bedrooms']*train_sales['bedrooms'], train_sales['bedrooms']*train_sales['bathrooms'],np.log(train_sales['sqft_living']),train_sales['lat']+train_sales['long']]
test_sales['bedrooms_squared'],test_sales['bed_bath_rooms'],test_sales['log_sqft_living'],test_sales['lat_plus_long'] = [test_sales['bedrooms']*test_sales['bedrooms'], test_sales['bedrooms']*test_sales['bathrooms'],np.log(test_sales['sqft_living']),test_sales['lat']+test_sales['long']]
train_sales.head()


<ipython-input-12-c4b0ea9ba42d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sales['bedrooms_squared'],train_sales['bed_bath_rooms'],train_sales['log_sqft_living'],train_sales['lat_plus_long'] = [train_sales['bedrooms']*train_sales['bedrooms'], train_sales['bedrooms']*train_sales['bathrooms'],np.log(train_sales['sqft_living']),train_sales['lat']+train_sales['long']]
<ipython-input-12-c4b0ea9ba42d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sales['bedrooms_squared'],test_sales['bed_bath_

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
16000,2561340020,20140804T000000,325000.0,3,1.75,1780,11096,1.0,0,0,...,0,98074,47.6170,-122.051,1780,10640,9,5.25,7.484369,-74.4340
11286,8598200070,20141208T000000,278000.0,2,2.50,1420,2229,2.0,0,0,...,0,98059,47.4871,-122.165,1500,2230,4,5.00,7.258412,-74.6779
3201,6788200931,20140520T000000,710000.0,2,1.00,1790,4000,1.0,0,0,...,0,98112,47.6405,-122.301,1310,4000,4,2.00,7.489971,-74.6605
11049,3023059012,20140910T000000,389900.0,4,1.00,1710,117176,1.5,0,0,...,0,98055,47.4497,-122.212,1940,12223,16,4.00,7.444249,-74.7623
9716,5683500030,20150320T000000,489000.0,4,1.00,1150,5217,1.5,0,0,...,0,98115,47.6806,-122.287,1220,5217,16,4.00,7.047517,-74.6064


Specifying different models of varying complexities

In [13]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [14]:
numeric_cols = train_sales.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove('id')
len(numeric_cols)

23

Redifining RSS function to calculate RSS using observed and predicted values

In [15]:
def get_RSS(output, predictions):
  RSS = ((output-predictions)**2).sum() 
  return(RSS)

Training all the models

In [16]:
X = train_sales[model_1_features]
y = train_sales['price']
model_1 = sm.OLS(y,X).fit()
predictions = model_1.predict(X)

In [17]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.855
Model:                            OLS   Adj. R-squared (uncentered):              0.855
Method:                 Least Squares   F-statistic:                          2.039e+04
Date:                Fri, 06 Nov 2020   Prob (F-statistic):                        0.00
Time:                        16:46:46   Log-Likelihood:                     -2.3933e+05
No. Observations:               17290   AIC:                                  4.787e+05
Df Residuals:                   17285   BIC:                                  4.787e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
sqft_living   293.5557      3.382     86.806      0.000     286.927     300.184
bedrooms    -5.218e+04   2501.600    -20.857      0.000   -5.71e+04   -4.73e+04
bathrooms    9423.3448   3804.545      2.477      0.013    1966.051    1.69e+04
lat          5.296e+05   1.35e+04     39.306      0.000    5.03e+05    5.56e+05
long         2.054e+05   5240.665     39.190      0.000    1.95e+05    2.16e+05
==============================================================================
Omnibus:                    13247.221   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           598071.263
Skew:                           3.271   Prob(JB):                         0.00
Kurtosis:                      31.060   Cond. No.                     1.74e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
X = train_sales[model_2_features]
y = train_sales['price']
model_2 = sm.OLS(y,X).fit()
predictions_2 = model_2.predict(X)

In [19]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.858
Model:                            OLS   Adj. R-squared (uncentered):              0.858
Method:                 Least Squares   F-statistic:                          1.736e+04
Date:                Fri, 06 Nov 2020   Prob (F-statistic):                        0.00
Time:                        16:46:46   Log-Likelihood:                     -2.3917e+05
No. Observations:               17290   AIC:                                  4.784e+05
Df Residuals:                   17284   BIC:                                  4.784e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
sqft_living      285.0825      3.385     84.226      0.000     278.448     291.717
bedrooms       -1.283e+05   4937.698    -25.983      0.000   -1.38e+05   -1.19e+05
bathrooms       -1.16e+05   7981.615    -14.530      0.000   -1.32e+05      -1e+05
lat               5.3e+05   1.34e+04     39.692      0.000    5.04e+05    5.56e+05
long            2.034e+05   5194.502     39.155      0.000    1.93e+05    2.14e+05
bed_bath_rooms  3.711e+04   2081.891     17.825      0.000     3.3e+04    4.12e+04
==============================================================================
Omnibus:                    12232.502   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           441011.443
Skew:                           2.965   Prob(JB):                         0.00
Kurtosis:                      27.021   Cond. No.                     1.74e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.74e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
X = train_sales[model_3_features]
y = train_sales['price']
model_3 = sm.OLS(y,X).fit()
predictions_3 = model_3.predict(X)

In [21]:
model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.867
Model:                            OLS   Adj. R-squared (uncentered):              0.867
Method:                 Least Squares   F-statistic:                          1.404e+04
Date:                Fri, 06 Nov 2020   Prob (F-statistic):                        0.00
Time:                        16:46:46   Log-Likelihood:                     -2.3861e+05
No. Observations:               17290   AIC:                                  4.772e+05
Df Residuals:                   17282   BIC:                                  4.773e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
sqft_living        538.4166      8.247     65.286      0.000     522.252     554.582
bedrooms         -1.394e+04   6760.671     -2.063      0.039   -2.72e+04    -692.543
bathrooms         7.201e+04   9507.927      7.573      0.000    5.34e+04    9.06e+04
lat               3.014e+05   6962.706     43.286      0.000    2.88e+05    3.15e+05
long             -6.196e+04   1150.819    -53.839      0.000   -6.42e+04   -5.97e+04
bed_bath_rooms   -1.025e+04   2454.084     -4.176      0.000   -1.51e+04   -5438.344
bedrooms_squared   952.4487    285.912      3.331      0.001     392.032    1512.865
log_sqft_living   -6.16e+05   1.86e+04    -33.091      0.000   -6.52e+05   -5.79e+05
lat_plus_long     2.394e+05   5986.928     39.992      0.000    2.28e+05    2.51e+05
==============================================================================
Omnibus:                    10122.076   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           281639.033
Skew:                           2.310   Prob(JB):                         0.00
Kurtosis:                      22.225   Cond. No.                     1.32e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.11e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [22]:
RSS_1 = get_RSS(train_sales['price'],predictions)
RSS_2 = get_RSS(train_sales['price'],predictions_2)
RSS_3 = get_RSS(train_sales['price'],predictions_3)

In [23]:
RSS = {'Model_1':RSS_1, 'Model_2':RSS_2, 'Model_3':RSS_3}
RSS

{'Model_1': 1067298280396458.1,
 'Model_2': 1048032346607161.6,
 'Model_3': 981772503767513.4}

In [24]:
print('The model with the lowest RSS is {} and has an RSS of {}'.format(min(RSS, key=lambda k: RSS[k]), min(RSS.values())))

The model with the lowest RSS is Model_3 and has an RSS of 981772503767513.4
